In [1]:
{"nombre": "Manzana", "fech_venc": "2024-12-01", "tipo": "Frutas", "cantidad": 50, "precio": 0.5}
{"nombre": "Arroz", "fech_venc": "2025-01-01", "tipo": "Cereales", "cantidad": 100, "precio": 1.2}
{"nombre": "Pollo", "fech_venc": "2024-11-20", "tipo": "Carnes", "cantidad": 30, "precio": 3.5}
{"nombre": "Lentejas", "fech_venc": "2025-06-15", "tipo": "Cereales", "cantidad": 200, "precio": 1.0}
{"nombre": "Leche", "fech_venc": "2024-12-10", "tipo": "Lácteos", "cantidad": 120, "precio": 1.5}
{"nombre": "Atún", "fech_venc": "2026-03-22", "tipo": "Conservas", "cantidad": 80, "precio": 2.0}
{"nombre": "Pera", "fech_venc": "2024-11-30", "tipo": "Frutas", "cantidad": 60, "precio": 0.7}
{"nombre": "Salchichón", "fech_venc": "2024-12-15", "tipo": "Carnes", "cantidad": 40, "precio": 4.0}
{"nombre": "Tomate en conserva", "fech_venc": "2025-12-01", "tipo": "Conservas", "cantidad": 150, "precio": 1.8}
{"nombre": "Yogur", "fech_venc": "2024-11-25", "tipo": "Lácteos", "cantidad": 200, "precio": 0.9}

{'nombre': 'Yogur',
 'fech_venc': '2024-11-25',
 'tipo': 'Lácteos',
 'cantidad': 200,
 'precio': 0.9}

In [2]:
import json
from datetime import datetime
from collections import defaultdict

# Clase Producto
class Producto:
    def __init__(self, nombre, fecha_vencimiento=None, tipo=None, cantidad=0, precio=0):
        self.nombre = nombre
        self.fecha_vencimiento = datetime.strptime(fecha_vencimiento, "%Y-%m-%d") if fecha_vencimiento else None
        self.tipo = tipo  # Carnes, Frutas, Cereales, Conservas, Lácteos
        self.cantidad = int(cantidad)  # Convertir a entero
        self.precio = float(precio)  # Convertir a float
        self.siguiente = None  # Enlace al siguiente producto

    @property
    def precio_total(self):
        return self.precio * self.cantidad  # Calcula el precio total

# Clase Nodo
class Nodo:
    def __init__(self, tipo):
        self.tipo = tipo
        self.producto = None
        self.hijo = None  # Primer subcategoría o producto
        self.hermano = None  # Siguiente nodo del mismo nivel

    def agregar_producto(self, nuevo_producto):
        if not self.producto:
            self.producto = nuevo_producto
        else:
            actual = self.producto
            anterior = None
            while actual:
                anterior = actual
                actual = actual.siguiente
            nuevo_producto.siguiente = actual
            if anterior:
                anterior.siguiente = nuevo_producto
            else:
                self.producto = nuevo_producto

    def agregar_hijo(self, nodo_hijo):
        if not self.hijo:
            self.hijo = nodo_hijo
        else:
            actual = self.hijo
            while actual.hermano:
                actual = actual.hermano
            actual.hermano = nodo_hijo

    def buscar_producto_por_tipo(self, tipo):
        if self.tipo == tipo:
            return self
        actual = self.hijo
        while actual:
            resultado = actual.buscar_producto_por_tipo(tipo)
            if resultado:
                return resultado
            actual = actual.hermano
        return None

    def buscar_producto_por_vencimiento(self, fecha):
        fecha_busqueda = datetime.strptime(fecha, "%Y-%m-%d")
        producto_actual = self.producto
        while producto_actual:
            if producto_actual.fecha_vencimiento <= fecha_busqueda:
                print(f"Producto vencido: {producto_actual.nombre}")
            producto_actual = producto_actual.siguiente
        hijo = self.hijo
        while hijo:
            hijo.buscar_producto_por_vencimiento(fecha)
            hijo = hijo.hermano

    def eliminar_productos_vencidos(self):
        fecha_actual = datetime.now()
        producto_actual = self.producto
        anterior = None
        while producto_actual:
            if producto_actual.fecha_vencimiento <= fecha_actual:
                if anterior:
                    anterior.siguiente = producto_actual.siguiente
                else:
                    self.producto = producto_actual.siguiente
            else:
                anterior = producto_actual
            producto_actual = producto_actual.siguiente
        hijo = self.hijo
        while hijo:
            hijo.eliminar_productos_vencidos()
            hijo = hijo.hermano

# Clase Arbol
class Arbol:
    def __init__(self, tipo_raiz):
        self.raiz = Nodo(tipo_raiz)
        self.contador_productos = 0  # Contador de productos en el árbol
        self.limite_productos = 10000  # Límite de productos que se pueden agregar

    def agregar_tipo(self, nodo_tipo):
        self.raiz.agregar_hijo(nodo_tipo)

    def agregar_producto_a_tipo(self, tipo, producto):
        if self.contador_productos >= self.limite_productos:
            print(f"Limite de productos alcanzado. Actualmente hay {self.contador_productos} productos en el almacén.")
            return
        nodo_tipo = self.raiz.buscar_producto_por_tipo(tipo)
        if nodo_tipo:
            nodo_tipo.agregar_producto(producto)
            self.contador_productos += 1
        else:
            print(f"Tipo de producto '{tipo}' no encontrado.")
    
    def buscar_por_tipo(self, tipo):
        nodo_tipo = self.raiz.buscar_producto_por_tipo(tipo)
        return nodo_tipo.producto if nodo_tipo else None

    def buscar_por_vencimiento(self, fecha):
        self.raiz.buscar_producto_por_vencimiento(fecha)

    def eliminar_productos_vencidos(self):
        self.raiz.eliminar_productos_vencidos()

    def calcular_suma_cantidad_por_tipo(self):
        cantidades = defaultdict(int)
        self._calcular_suma_cantidad_por_tipo_recursivo(self.raiz, cantidades)
        return dict(cantidades)

    def _calcular_suma_cantidad_por_tipo_recursivo(self, nodo, cantidades):
        if nodo.producto:
            producto_actual = nodo.producto
            while producto_actual:
                cantidades[nodo.tipo] += producto_actual.cantidad
                producto_actual = producto_actual.siguiente
        if nodo.hijo:
            self._calcular_suma_cantidad_por_tipo_recursivo(nodo.hijo, cantidades)
        if nodo.hermano:
            self._calcular_suma_cantidad_por_tipo_recursivo(nodo.hermano, cantidades)

    def calcular_suma_cantidad_por_nombre(self):
        nombres = defaultdict(int)
        self._calcular_suma_cantidad_por_nombre_recursivo(self.raiz, nombres)
        return dict(nombres)

    def _calcular_suma_cantidad_por_nombre_recursivo(self, nodo, nombres):
        if nodo.producto:
            producto_actual = nodo.producto
            while producto_actual:
                nombres[producto_actual.nombre] += producto_actual.cantidad
                producto_actual = producto_actual.siguiente
        if nodo.hijo:
            self._calcular_suma_cantidad_por_nombre_recursivo(nodo.hijo, nombres)
        if nodo.hermano:
            self._calcular_suma_cantidad_por_nombre_recursivo(nodo.hermano, nombres)

    def calcular_suma_cantidad_por_tipo_y_precio(self):
        tipo_precio = defaultdict(float)
        self._calcular_suma_cantidad_por_tipo_y_precio_recursivo(self.raiz, tipo_precio)
        return dict(tipo_precio)

    def _calcular_suma_cantidad_por_tipo_y_precio_recursivo(self, nodo, tipo_precio):
        if nodo.producto:
            producto_actual = nodo.producto
            while producto_actual:
                tipo_precio[nodo.tipo] += producto_actual.precio_total
                producto_actual = producto_actual.siguiente
        if nodo.hijo:
            self._calcular_suma_cantidad_por_tipo_y_precio_recursivo(nodo.hijo, tipo_precio)
        if nodo.hermano:
            self._calcular_suma_cantidad_por_tipo_y_precio_recursivo(nodo.hermano, tipo_precio)

In [3]:
def imprimir_productos_por_tipo(arbol, tipo):
        nodo_tipo = arbol.raiz.buscar_producto_por_tipo(tipo)
        if nodo_tipo:
            print(f"Productos de tipo '{tipo}':")
            producto_actual = nodo_tipo.producto
            while producto_actual:
                print(f"Nombre: {producto_actual.nombre}, Vencimiento: {producto_actual.fecha_vencimiento}, Cantidad: {producto_actual.cantidad}, Precio unitario: {producto_actual.precio}, Precio total: {producto_actual.precio * producto_actual.cantidad}")
                producto_actual = producto_actual.siguiente
        else:
            print(f"Tipo de producto '{tipo}' no encontrado.")

In [4]:
# Función para cargar datos desde archivo
def cargar_datos_desde_archivo(nombre_archivo, arbol):
    with open(nombre_archivo, 'r', encoding='utf-8') as archivo:  # Asegurarse de usar UTF-8 para leer el archivo
        lineas = archivo.readlines()
        for linea in lineas:
            datos = json.loads(linea.strip())
            producto = Producto(
                nombre=datos["nombre"],
                fecha_vencimiento=datos.get("fech_venc"),
                tipo=datos["tipo"],  # Carnes, Frutas, Cereales, Conservas, Lácteos
                cantidad=int(datos["cantidad"]),  # Convertir a entero
                precio=float(datos["precio"])  # Precio del producto
            )
            # Verificar si el tipo de producto ya existe, y si no, agregarlo
            nodo_tipo = arbol.raiz.buscar_producto_por_tipo(producto.tipo)
            if not nodo_tipo:
                nodo_tipo = Nodo(producto.tipo)
                arbol.agregar_tipo(nodo_tipo)
            arbol.agregar_producto_a_tipo(producto.tipo, producto)

In [5]:
arbol_productos = Arbol('Inventario')

In [6]:
cargar_datos_desde_archivo('inventario.txt', arbol_productos)

In [7]:
imprimir_productos_por_tipo(arbol_productos, "Carnes")

Productos de tipo 'Carnes':
Nombre: Pollo, Vencimiento: 2024-11-20 00:00:00, Cantidad: 30, Precio unitario: 3.5, Precio total: 105.0
Nombre: Salchichón, Vencimiento: 2024-12-15 00:00:00, Cantidad: 40, Precio unitario: 4.0, Precio total: 160.0


In [8]:
arbol_productos.eliminar_productos_vencidos()

In [9]:
# Método para contar todos los productos en el árbol
def contar_productos(arbol):
    cantidad_total = 0
    
    # Recorrer recursivamente el árbol y contar los productos
    def contar_nodo(nodo):
        nonlocal cantidad_total
        if nodo.producto:
            producto_actual = nodo.producto
            while producto_actual:
                cantidad_total += producto_actual.cantidad
                producto_actual = producto_actual.siguiente
        if nodo.hijo:
            contar_nodo(nodo.hijo)
        if nodo.hermano:
            contar_nodo(nodo.hermano)
    
    contar_nodo(arbol.raiz)
    return cantidad_total

# Método para agregar un producto
def agregar_producto(arbol, nombre, fecha_vencimiento, tipo, cantidad, precio):
    # Contar los productos actuales en el árbol
    cantidad_actual = contar_productos(arbol)
    
    # Verificar si no se ha excedido el límite de productos
    if cantidad_actual + cantidad > 10000:
        print(f"El límite de productos ({cantidad_actual}) ha sido alcanzado. No se puede agregar más productos.")
        return
    
    # Crear el producto a partir de los datos proporcionados
    producto = Producto(
        nombre=nombre,
        fecha_vencimiento=fecha_vencimiento,
        tipo=tipo,
        cantidad=int(cantidad),  # Convertir a entero
        precio=float(precio)  # Convertir el precio a tipo float
    )
    
    # Verificar si el tipo de producto ya existe en el árbol
    nodo_tipo = arbol.raiz.buscar_producto_por_tipo(tipo)
    
    # Si el nodo del tipo no existe, crear un nuevo nodo
    if not nodo_tipo:
        nodo_tipo = Nodo(tipo)
        arbol.agregar_tipo(nodo_tipo)
    
    # Agregar el producto al tipo correspondiente
    arbol.agregar_producto_a_tipo(tipo, producto)
    print(f"Producto '{nombre}' agregado correctamente.")


In [10]:
contar_productos(arbol_productos)

800

In [11]:
# Supongamos que arbol_productos es tu árbol de productos
agregar_producto(arbol_productos, "Naranja", "2024-12-15", "Frutas", 4000, 0.6)

Producto 'Naranja' agregado correctamente.


In [17]:
from collections import deque
from datetime import datetime

class Cola:
    def __init__(self):
        self.cola = deque()  
        self.cola_vencidos = deque()  

    def agregar(self, lote_producto):
        self.cola.append(lote_producto) 

    def retirar(self):
        if len(self.cola) > 0: 
            productos_no_vencidos = []
            while len(self.cola) > 0:
                lote = self.cola.popleft()  
                if self._producto_vencido(lote):
                    print(f"Lote de {lote['nombre']} eliminado debido a su fecha de vencimiento.")
                    self.cola_vencidos.append(lote) 
                else:
                    productos_no_vencidos.append(lote)
            for lote in productos_no_vencidos:
                self.cola.append(lote)
                print(f"Lote de {lote['nombre']} no está vencido.")
            return None  
        else:
            print("La cola está vacía.")
            return None
    def esta_vacia(self):
        return len(self.cola) == 0
    def _producto_vencido(self, lote_producto):
        fecha_vencimiento = datetime.strptime(lote_producto['fech_venc'], "%Y-%m-%d")
        return fecha_vencimiento < datetime.now()
    def mostrar_productos(self):
        if self.esta_vacia():
            print("No hay productos en la cola.")
        else:
            for lote in self.cola:
                print(f"Lote de {lote['nombre']}, Fecha de vencimiento: {lote['fech_venc']}, Cantidad: {lote['cantidad']}")
    def mostrar_vencidos(self):
        if len(self.cola_vencidos) == 0:
            print("No hay productos vencidos.")
        else:
            for lote in self.cola_vencidos:
                print(f"Lote vencido de {lote['nombre']}, Fecha de vencimiento: {lote['fech_venc']}, Cantidad: {lote['cantidad']}")
almacen = Cola()

almacen.agregar({"nombre": "Manzanas", "fech_venc": "2023-11-20", "tipo": "Frutas", "cantidad": 100, "precio": 1.2})
almacen.agregar({"nombre": "Pollo", "fech_venc": "2024-12-15", "tipo": "Carnes", "cantidad": 50, "precio": 3.5})
almacen.agregar({"nombre": "Leche", "fech_venc": "2023-12-15", "tipo": "Lácteos", "cantidad": 200, "precio": 1.0})

almacen.retirar()
almacen.mostrar_productos()
almacen.mostrar_vencidos()

Lote de Manzanas eliminado debido a su fecha de vencimiento.
Lote de Leche eliminado debido a su fecha de vencimiento.
Lote de Pollo no está vencido.
Lote de Pollo, Fecha de vencimiento: 2024-12-15, Cantidad: 50
Lote vencido de Manzanas, Fecha de vencimiento: 2023-11-20, Cantidad: 100
Lote vencido de Leche, Fecha de vencimiento: 2023-12-15, Cantidad: 200
